<a href="https://colab.research.google.com/github/shravya004/Ai-Weed-Sprayer/blob/main/FINAL_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 1. Install Libraries ---
print("🚀 Installing necessary libraries...")
!pip install -q kaggle
!pip install -q qiskit[machine-learning]
!pip install -q qiskit-aer

# --- 2. Configure Kaggle API ---
print("\n🔑 Please upload your kaggle.json file")
from google.colab import files
files.upload() # Upload the kaggle.json file you downloaded from your Kaggle account

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("\n✅ Kaggle API configured.")

# --- 3. Download and Unzip Dataset ---
print("\nDownloading and unzipping the PlantVillage dataset...")
!kaggle datasets download -d emmarex/plantdisease
!unzip -q plantdisease.zip
print("\n✅ Dataset is ready.")

🚀 Installing necessary libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 100.3 MB/s eta 0:00:00

🔑 Please upload your kaggle.json file


Saving kaggle.json to kaggle.json

✅ Kaggle API configured.

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 94% 619M/658M [00:03<00:00, 203MB/s]
100% 658M/658M [00:03<00:00, 211MB/s]

✅ Dataset is ready.


In [2]:
!pip install qiskit-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 11.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
  Attempting uninstall: qiskit
    Found existing installation: qiskit 2.2.1
    Uninstalling qiskit-2.2.1:
      Successfully uninstalled qiskit-2.2.1


In [9]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- Parameters ---
IMAGE_SIZE = 32 # Using 32x32 for efficiency
DATA_DIR = 'PlantVillage/'

# --- 1. Load Data for ALL Classes ---
print("💾 Loading all images from the dataset...")
data = []
labels = []
class_names = sorted(os.listdir(DATA_DIR)) # Get all class folder names
num_classes = len(class_names)

# Create a mapping from class name to integer label
class_to_int = {name: i for i, name in enumerate(class_names)}

for class_name in class_names:
    class_path = os.path.join(DATA_DIR, class_name)
    class_label = class_to_int[class_name]

    for img_name in os.listdir(class_path):
        try:
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            data.append(resized_image)
            labels.append(class_label)
        except Exception as e:
            pass # Skip corrupted files

# Convert to numpy arrays
X = np.array(data)
y = np.array(labels)

# --- 2. One-Hot Encode the Labels ---
# Example: If a label is '2' and there are 15 classes, it becomes [0, 0, 1, 0, ..., 0]
y = tf.keras.utils.to_categorical(y, num_classes=num_classes)

# --- 3. Preprocess and Split the Data ---
X_flat = X.reshape(len(X), -1) # Flatten for the scaler
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the scaled data back into images for the CNN
X_train_cnn = X_train_scaled.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
X_test_cnn = X_test_scaled.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)

print(f"\n✅ Data loading complete.")
print(f"Total classes found: {num_classes}")
print(f"Shape of training data for CNN: {X_train_cnn.shape}")
print(f"Shape of training labels: {y_train.shape}")

💾 Loading all images from the dataset...

✅ Data loading complete.
Total classes found: 15
Shape of training data for CNN: (16510, 32, 32, 1)
Shape of training labels: (16510, 15)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# --- 1. Define the Updated CNN Architecture ---
print("\n🧠 Building the multi-class CNN model...")
cnn_model_multi = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'), # Increased dense layer size for more complexity
    Dropout(0.5),

    # --- CRUCIAL CHANGES HERE ---
    # The output layer must have one neuron for each class
    Dense(num_classes, activation='softmax')
])

# --- 2. Compile the Model with Categorical Loss ---
cnn_model_multi.compile(optimizer='adam',
                        loss='categorical_crossentropy', # <-- Use this for one-hot labels
                        metrics=['accuracy'])

cnn_model_multi.summary()

# --- 3. Train the Model ---
print("\n🏋️ Training the multi-class CNN model...")
history = cnn_model_multi.fit(
    X_train_cnn,
    y_train,
    epochs=25, # Increased epochs slightly as the task is harder
    validation_data=(X_test_cnn, y_test), # Evaluate on test data each epoch
    verbose=1
)

# --- 4. Evaluate the Final Model ---
print("\n📊 Evaluating the final multi-class CNN...")
loss, accuracy = cnn_model_multi.evaluate(X_test_cnn, y_test, verbose=0)
print(f"\n✅ Final CNN Test Accuracy on the full dataset: {accuracy:.4f}")


🧠 Building the multi-class CNN model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 315,791 (1.20 MB)

 Trainable params: 315,791 (1.20 MB)

 Non-trainable params: 0 (0.00 B)


🏋️ Training the multi-class CNN model...
Epoch 1/25
516/516 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.3145 - loss: 2.1349 - val_accuracy: 0.5906 - val_loss: 1.2813
Epoch 2/25
516/516 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - accuracy: 0.5563 - loss: 1.3787 - val_accuracy: 0.6536 - val_loss: 1.0508
Epoch 3/25
516/516 ━━━━━━━━━━━━━━━━━━━━ 22s 43ms/step - accuracy: 0.6183 - loss: 1.1673 - val_accuracy: 0.7028 - val_loss: 0.9335
Epoch 4/25
516/516 ━━━━━━━━━━━━━━━━━━━━ 22s 43ms/step - accuracy: 0.6692 - loss: 0.9848 - val_accuracy: 0.7171 - val_loss: 0.8614
Epoch 5/25
516/516 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.7033 - loss: 0.8836 - val_accuracy: 0.7280 - val_loss: 0.8152
Epoch 6/25
516/516 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - accuracy: 0.7244 - loss: 0.8151 - val_accuracy: 0.7442 - val_loss: 0.7720
Epoch 7/25
516/516 ━━━━━━━━━━━━━━━━━━━━ 24s 47ms/step - accuracy: 0.7433 - loss: 0.7414 - val_accuracy: 0.7379 - val_loss: 0.7728
Epoch 8/25
516/516 ━━━━━━━━━━━━━━━━━━━━ 23s 44ms

In [3]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle

# --- Parameters ---
IMAGE_SIZE = 32 # Resize images to a smaller 32x32 size for efficiency
DATA_DIR = 'PlantVillage/'

# --- Select ONLY two classes for our binary problem ---
CLASS_HEALTHY = 'Pepper__bell___healthy'
CLASS_DISEASED = 'Pepper__bell___Bacterial_spot'
SELECTED_CLASSES = [CLASS_HEALTHY, CLASS_DISEASED]

# --- Load the Data ---
print(f"Loading images for the following two classes:\n1. {CLASS_HEALTHY}\n2. {CLASS_DISEASED}")

data = []
labels = []

for i, class_name in enumerate(SELECTED_CLASSES):
    path = os.path.join(DATA_DIR, class_name)
    for img_name in os.listdir(path):
        try:
            img_path = os.path.join(path, img_name)
            # Load image in grayscale and resize
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

            # Flatten the image into a 1D vector and add to our lists
            data.append(resized_image.flatten())
            labels.append(i) # Label 0 for healthy, 1 for diseased
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")

# Convert to numpy arrays
X = np.array(data)
y = np.array(labels)

# Shuffle the data to ensure randomness
X, y = shuffle(X, y, random_state=42)

print(f"\n✅ Data loading complete.")
print(f"Shape of our feature data (X): {X.shape}")
print(f"Shape of our labels (y): {y.shape}")

Loading images for the following two classes:
1. Pepper__bell___healthy
2. Pepper__bell___Bacterial_spot

✅ Data loading complete.
Shape of our feature data (X): (2475, 1024)
Shape of our labels (y): (2475,)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# --- 1. Split Data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 2. Scale Data ---
# Fit on the training data and transform both train and test data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 3. Apply PCA for Dimensionality Reduction ---
N_COMPONENTS = 8 # The number of features to reduce to
print(f"\n🔬 Applying PCA to reduce features to {N_COMPONENTS} dimensions...")

pca = PCA(n_components=N_COMPONENTS)
pca.fit(X_train_scaled)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"\n✅ PCA complete.")
print(f"Shape of training data after PCA: {X_train_pca.shape}")


🔬 Applying PCA to reduce features to 8 dimensions...

✅ PCA complete.
Shape of training data after PCA: (1980, 8)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# --- 1. Reshape Data for CNN ---
# CNNs need image data in a specific shape: (samples, height, width, channels)
# Our images are 32x32 and grayscale (1 channel).
X_train_cnn = X_train_scaled.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
X_test_cnn = X_test_scaled.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)

print(f"Reshaped training data for CNN: {X_train_cnn.shape}")

# --- 2. Define the CNN Architecture ---
print("\n🧠 Building the CNN model...")
cnn_model = Sequential([
    # First convolutional layer
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
    MaxPooling2D((2, 2)),

    # Second convolutional layer
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Flatten the results to feed into a dense layer
    Flatten(),

    # Dense layer for classification
    Dense(64, activation='relu'),
    Dropout(0.5), # Dropout helps prevent overfitting

    # Output layer: 1 neuron with a sigmoid activation for binary classification
    Dense(1, activation='sigmoid')
])

# --- 3. Compile the Model ---
cnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

cnn_model.summary()

# --- 4. Train the Model ---
print("\n🏋️ Training the CNN model...")
history = cnn_model.fit(
    X_train_cnn,
    y_train,
    epochs=20, # 20 epochs is a good starting point
    validation_split=0.2, # Use 20% of training data for validation
    verbose=1 # Show progress
)

# --- 5. Evaluate the Model on the Test Set ---
print("\n📊 Evaluating the CNN on the test set...")
loss, accuracy = cnn_model.evaluate(X_test_cnn, y_test, verbose=0)

print(f"\n✅ CNN Test Accuracy: {accuracy:.4f}")

# --- You can now compare this with your other models ---

Reshaped training data for CNN: (1980, 32, 32, 1)

🧠 Building the CNN model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 166,401 (650.00 KB)

 Trainable params: 166,401 (650.00 KB)

 Non-trainable params: 0 (0.00 B)


🏋️ Training the CNN model...
Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.6696 - loss: 0.6073 - val_accuracy: 0.7854 - val_loss: 0.4329
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.7911 - loss: 0.4325 - val_accuracy: 0.8258 - val_loss: 0.3911
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.8346 - loss: 0.3765 - val_accuracy: 0.8510 - val_loss: 0.3474
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.8568 - loss: 0.3267 - val_accuracy: 0.8535 - val_loss: 0.3496
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.8941 - loss: 0.2656 - val_accuracy: 0.8763 - val_loss: 0.2968
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.8988 - loss: 0.2662 - val_accuracy: 0.8687 - val_loss: 0.2989
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9136 - loss: 0.2284 - val_accuracy: 0.8687 - val_loss: 0.2886
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9237 - loss: 0.2

In [8]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel # <-- IMPORT THIS
from qiskit_machine_learning.algorithms import QSVC

# --- 1. Classical SVM (Baseline) ---
print("\n🤖 Training classical SVM as a baseline...")
svc = SVC()
svc.fit(X_train_pca, y_train)
svc_score = svc.score(X_test_pca, y_test)
print(f"\nClassical SVM Accuracy: {svc_score:.4f}")

print("\nClassical SVM Classification Report:")
y_pred_svc = svc.predict(X_test_pca)
print(classification_report(y_test, y_pred_svc, target_names=SELECTED_CLASSES))


# --- 2. Quantum SVM (QSVM) ---
# Note: QSVM is slow, so we often train on a smaller subset of the training data.
TRAIN_SUBSET_SIZE = 250
X_train_q = X_train_pca[:TRAIN_SUBSET_SIZE]
y_train_q = y_train[:TRAIN_SUBSET_SIZE]

print(f"\n⚛️  Training QSVM on a subset of {TRAIN_SUBSET_SIZE} samples... (This may take a few minutes)")

# Define the quantum feature map
feature_map = ZZFeatureMap(feature_dimension=N_COMPONENTS, reps=2, entanglement='linear')

# --- THIS IS THE UPDATED PART ---
# Create the quantum kernel from the feature map
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

# Create and configure the QSVC using the kernel
qsvc = QSVC(quantum_kernel=quantum_kernel)
# ---------------------------------

qsvc.fit(X_train_q, y_train_q)

# Evaluate the QSVM on the full test set
qsvc_score = qsvc.score(X_test_pca, y_test)
print(f"\nQSVM Accuracy: {qsvc_score:.4f}")

print("\nQSVM Classification Report:")
y_pred_qsvc = qsvc.predict(X_test_pca)
print(classification_report(y_test, y_pred_qsvc, target_names=SELECTED_CLASSES))

# --- Compare with the CNN model ---
print(f"\nClassical SVM Accuracy: {svc_score:.4f}")
print(f"QSVM Accuracy: {qsvc_score:.4f}")
print(f"CNN Test Accuracy: {accuracy:.4f}")


🤖 Training classical SVM as a baseline...

Classical SVM Accuracy: 0.8263

Classical SVM Classification Report:
                               precision    recall  f1-score   support

       Pepper__bell___healthy       0.84      0.88      0.86       294
Pepper__bell___Bacterial_spot       0.81      0.75      0.78       201

                     accuracy                           0.83       495
                    macro avg       0.82      0.81      0.82       495
                 weighted avg       0.83      0.83      0.83       495


⚛️  Training QSVM on a subset of 250 samples... (This may take a few minutes)


KeyboardInterrupt: 

In [7]:
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import numpy as np

# --- NOTE: We now use ALL the data (X, y) to train the final model ---
# This ensures the model learns from every image we have.

# 1. Scale the entire dataset
final_scaler = StandardScaler().fit(X)
X_scaled = final_scaler.transform(X)

# 2. Apply PCA on the entire dataset
N_COMPONENTS = 8
final_pca = PCA(n_components=N_COMPONENTS)
X_pca = final_pca.fit_transform(X_scaled)

# 3. Train the final SVM model
print("💪 Training the final SVM model on all available data...")
final_svm = SVC(probability=True) # probability=True is useful for getting confidence scores
final_svm.fit(X_pca, y)

# 4. Save the model and the preprocessors
joblib.dump(final_svm, 'svm_model.joblib')
joblib.dump(final_scaler, 'scaler.joblib')
joblib.dump(final_pca, 'pca.joblib')

print("\n✅ Models and preprocessors saved!")
print("Download the following files from Colab: svm_model.joblib, scaler.joblib, pca.joblib")

💪 Training the final SVM model on all available data...

✅ Models and preprocessors saved!
Download the following files from Colab: svm_model.joblib, scaler.joblib, pca.joblib
